In [2]:
import requests
import os
import warnings
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray as xr
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from rasterio.warp import Resampling
import matplotlib.pyplot as plt
import zarr
from dotenv import load_dotenv
import os
import ipywidgets as widgets
from ipywidgets import interact
from traitlets import link
from localtileserver import get_leaflet_tile_layer, TileClient
from tempfile import NamedTemporaryFile
import warnings
import ssl
from zarr_visualization.access_bucket import AccessBucket

warnings.filterwarnings("ignore")
load_dotenv()
%load_ext autoreload
%autoreload 2

In [8]:
bucket_name = 'tobfer'
aws_access_key_id = os.environ.get('aws_access_key_id')
aws_secret_access_key = os.environ.get('aws_secret_access_key')
endpoint_url = os.environ.get('endpoint_url')

a = AccessBucket(aws_access_key_id=aws_access_key_id,
                 aws_secret_access_key=aws_secret_access_key,
                 endpoint_url=endpoint_url,
                 bucket=bucket_name)

In [9]:
files = a.ls(bucket_folder='')

KeyboardInterrupt: 

In [18]:
files

['T1d', 'T1m']

In [19]:
file_objects = a.ls(bucket_folder='')


['T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d', 'T1d

In [12]:
list_of_variables = [item.split('/')[-1] for item in fs.ls(bucket_name)]

TypeError: AioSession._create_client() got an unexpected keyword argument 'context'

In [1]:
zarr_viz = ZarrVisualization(bucket_name='tobfer')


NameError: name 'ZarrVisualization' is not defined

In [3]:
variable_select = widgets.Dropdown(
    options=zarr_viz.list_of_variables,
    value=zarr_viz.list_of_variables[0], 
    description='Variable:'
)

In [8]:
limits = [0,0,0,0,0]
open_button = widgets.Button(description="Open Zarr")
sel_button = widgets.HBox()
show_data = widgets.HBox()

lat_min_container = widgets.HBox()
lat_max_container = widgets.HBox()
lon_min_container = widgets.HBox()
lon_max_container = widgets.HBox()
data_path = widgets.HBox()
data_information = widgets.HBox()
time_counter_container = widgets.HBox()

loading_spinner = widgets.HTML(value="Loading...") 
loading_container = widgets.VBox()


m = Map(center=[40, 0], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')

def plot_data_old(ds):
    print(ds)
    l = ds.leaflet.plot(m, colormap=plt.cm.terrain)
    print('l',l)

def plot_data(ds):
    loading_container.children = [loading_spinner]

    temp_file = NamedTemporaryFile(suffix='.tif', delete=False)
    ds.rio.to_raster(temp_file.name)
    temp_file.close()
    client = TileClient(temp_file.name)
    t = get_leaflet_tile_layer(source=client, colormap='RdYlBu_r')
    m.add(t)
    loading_container.children = []


def sel_zarr_data(ds, limits):
    loading_container.children = [loading_spinner]
    ds = ds.sel(y=slice(limits[0], limits[1]))
    ds = ds.sel(x=slice(limits[2], limits[3]))
    if len(limits) == 5:
        ds = ds.sel(time_counter=limits[4])
    ds.attrs['scale_factor'] = 1.0
    ds.attrs['add_offset'] = 0.0
    ds = ds.assign_coords(band=1)
    ds = ds.drop_vars(['nav_lat', 'nav_lon'])
    data_information.children = [widgets.Output()]
    with data_information.children[0]:
        print(ds)

    loading_container.children = []
    show_data.children = [widgets.Button(description="Plot Data")]
    show_data.children[0].on_click(lambda b: plot_data(ds))
    
def open_zarr_file(data_path):
    ds = xr.open_zarr(data_path)
    nav_lat = ds.nav_lat.values
    new_y = np.linspace(nav_lat.min(), nav_lat.max(), num=nav_lat.shape[0])
    nav_lon = ds.nav_lon.values
    # new_x = np.linspace(nav_lon.min(), nav_lon.max(), num=nav_lon.shape[1])
    new_x = nav_lon[1000]
    ds = ds.assign_coords(y=new_y, x=new_x)
    ds = ds.sortby('x')
    ds = ds.sortby('y')
    if "x" not in ds.dims and "y" not in ds.dims:
        latitude_var_name = "lat"
        longitude_var_name = "lon"
        if "latitude" in ds.dims:
            latitude_var_name = "latitude"
        if "longitude" in ds.dims:
            longitude_var_name = "longitude"
        ds = ds.rename({latitude_var_name: "y", longitude_var_name: "x"})
    if "time_counter" in ds.dims:
        ds = ds.transpose("time_counter", "y", "x")
    else:
        ds = ds.transpose("y", "x")
    if (ds.x > 180).any():
        ds = ds.assign_coords(x=(da.x + 180) % 360 - 180)
        ds = ds.sortby(da.x)
    crs = ds.rio.crs or "epsg:4326"
    ds.rio.write_crs(crs, inplace=True)
    return ds

def update_limits(change):
    limits[0] = lat_min_container.children[0].value
    limits[1] = lat_max_container.children[0].value
    limits[2] = lon_min_container.children[0].value
    limits[3] = lon_max_container.children[0].value
    if len(limits) == 5:
        limits[4] = time_counter_container.children[0].value

def process_zarr_file(subdata_select):
    loading_container.children = [loading_spinner]

    data_path = f"{remote_options['endpoint_url']}{bucket_name}/{variable_select.value}/{subdata_select.value}"
    print(data_path)

    ds = open_zarr_file(data_path)
    ds = ds[subdata_select.value[:-5]]
    lat_min_container.children = [
        widgets.FloatSlider(
            value=40,
            min=-89,
            max=89,
            description='Min Lat:',
            continuous_update=False
        )
    ]

    lat_max_container.children = [widgets.FloatSlider(
        value=60,
        min=-89,
        max=89,
        description='Max Lat:',
        continuous_update=False
    )]
    lon_min_container.children = [widgets.FloatSlider(
        value=-54,
        min=-179,
        max=179,
        description='Min Lon:',
        continuous_update=False
    )]
    
    lon_max_container.children = [widgets.FloatSlider(
        value=-37,
        min=-179,
        max=179,
        description='Max Lon:',
        continuous_update=False
    )]

    if 'time_counter' in list(ds.coords):
        time_values = ds.time_counter.values
        time_counter_container.children = [
            widgets.Dropdown(
                options=time_values,
                value= time_values[0], 
                description='Time Counter:'
            )
        ]
        time_counter_container.children[0].observe(update_limits, names='value')
    update_limits('')
    lat_min_container.children[0].observe(update_limits, names='value')
    lat_max_container.children[0].observe(update_limits, names='value')
    lon_min_container.children[0].observe(update_limits, names='value')
    lon_max_container.children[0].observe(update_limits, names='value')

    loading_container.children = []
    sel_button.children = [widgets.Button(description="Sel Data")]
    sel_button.children[0].on_click(lambda b: sel_zarr_data(ds, limits))


def update_data_path(change):
    data_path.children = [widgets.Output()]
    with data_path.children[0]:
        print(f"{remote_options['endpoint_url']}{bucket_name}/{variable_select.value}/{change['new']}")


@interact
def select_data(
    variable_select=variable_select):

    list_of_variables_subdata = fs.ls(f"{bucket_name}/{variable_select}")
    list_of_variables_subdata = [item.split('/')[-1] for item in list_of_variables_subdata]
    subdata_select = widgets.Dropdown(
        options=list_of_variables_subdata,
        value=list_of_variables_subdata[0],  
        description='Files:'
    )
    subdata_select.observe(update_data_path, names='value')
    display(subdata_select, data_path)
    display(open_button)

    display(time_counter_container)  
    display(lat_min_container)  
    display(lat_max_container)
    display(lon_min_container)
    display(lon_max_container)
    display(sel_button)
    display(data_information)
    display(show_data)
    display(loading_container)
    open_button.on_click(lambda b: process_zarr_file(subdata_select))
    
m

interactive(children=(Dropdown(description='Variable:', options=('T1d', 'T1m', 'T1y', 'T5d', 'U1d', 'U1m', 'U1…

Map(center=[40, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [12]:
ds = xr.open_zarr('https://noc-msm-o.s3-ext.jc.rl.ac.uk/tobfer/V1m/vo.zarr')